In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import numpy as np
%matplotlib inline

In [131]:
df = pd.read_excel('/home/dariodg/Desktop/free/Galpao/ComissionApp/docs/comissabril2019.XLS', )

In [132]:
df.columns

Index(['turma', 'titulo', 'vencimento', 'sacado', 'recebido', 'credito',
       'descritivo'],
      dtype='object')

## Turmas - Adicionando "sem turma" aos valores vazios

In [133]:
turmas = df['turma']

In [134]:
turmas.describe()

count                          218
unique                          35
top       2019 AÉREOS 2x - TURMA 3
freq                            18
Name: turma, dtype: object

In [135]:
print(type(turmas[0]) == float) # Valor sem turma é considerado false, tipo float

True


In [136]:
def fill_sem_turma(value):
    if type(value) == float:
        return 'sem_turma'
    return value

In [137]:
turma2 = turmas.apply(fill_sem_turma)

In [138]:
turma2.head()

0                      sem_turma
1    2019 AC - ADOLESC - TURMA 1
2    2019 AC - ADOLESC - TURMA 1
3    2019 AC - ADOLESC - TURMA 1
4    2019 AC - ADOLESC - TURMA 1
Name: turma, dtype: object

In [139]:
tuple(df.head(0))

('turma',
 'titulo',
 'vencimento',
 'sacado',
 'recebido',
 'credito',
 'descritivo')

In [140]:
type(df.head(0))

pandas.core.frame.DataFrame

In [141]:
descritivo = df['descritivo']

In [142]:
descritivo.head()

0    parcela de anuidade - 3ª/12             R$ 435...
1    TAXA PROD - 3ª/12         R$ 435,00\nARTE CIRC...
2    ARTE CIRC  ADOL 1 - 3ª/12 R$ 435,00\nTAXA PROD...
3    TX ADESÃO - 1ª/1          R$ 220,00\nTAXA PROD...
4    ARTE CIRC  ADOL 1 - 3ª/12 R$ 435,00\nTAXA PROD...
Name: descritivo, dtype: object

In [143]:
def strip_n_split(value):
    value = value.strip()
    value = value.split('\n')
    return value

In [144]:
descritivo2 = descritivo.apply(strip_n_split)

In [145]:
descritivo2[0]

['parcela de anuidade - 3ª/12             R$ 435,00',
 'Valor de produção - AÉR INI PAL - 3ª/12 R$  16,66']

In [146]:
descritivo2

0      [parcela de anuidade - 3ª/12             R$ 43...
1      [TAXA PROD - 3ª/12         R$ 435,00, ARTE CIR...
2      [ARTE CIRC  ADOL 1 - 3ª/12 R$ 435,00, TAXA PRO...
3      [TX ADESÃO - 1ª/1          R$ 220,00, TAXA PRO...
4      [ARTE CIRC  ADOL 1 - 3ª/12 R$ 435,00, TAXA PRO...
                             ...                        
214    [ARTE CIRC  ADOL 1 - 8ª/12 R$ 207,50, Taxa pro...
215    [ARTE CIRC  ADOL 1 - 9ª/12 R$ 26,23, Taxa prod...
216    [Multas a receber        R$  37,00, AÉR 2X - A...
217    [Multas a receber        R$  40,00, BRINCADEIR...
218                 [INICIAÇÃO CRI 1 - 11ª/12 R$ 275,00]
Name: descritivo, Length: 219, dtype: object

In [147]:
goal = ['parcela de anuidade - 3ª/12             R$ 435,00', 'Valor de produção - AÉR INI PAL - 3ª/12 R$  16,66']

In [148]:
descritivo2[0] == goal

True

In [149]:
descritivo2[3]

['TX ADESÃO - 1ª/1          R$ 220,00',
 'TAXA PROD - 1ª/10         R$  20,00',
 'ARTE CIRC  ADOL 1 - 3ª/12 R$ 435,00']

In [150]:
df['sacado'][3]

'Giulia Marandino Diaz'

In [151]:
descritivo2[0].str.split(n=1)

AttributeError: 'list' object has no attribute 'str'

In [152]:
def get_match(row, pattern):
    """ 
    Select a value from a list that correspond to a given value, or
    returns empty if no match was found.
    PS: Assign the result to a new variable. Dont override the original
     """
    for value in row:
        if pattern in value:
            return value
    return ''

In [118]:
adesao = descritivo2.apply(get_match, args=('ADES',))

In [119]:
def strip_n_split(row, criteria):
    """Remove white spaces and split new lines, forming a list."""
    return row.strip().split(criteria)

In [120]:
def info_n_amount(pd_serie):
    df = pd_serie.str.split(pat=r'R\$', expand=True)
    return df

In [121]:
df = info_n_amount(adesao)

In [130]:
df1 = df[0].apply(strip_n_split)

TypeError: strip_n_split() missing 1 required positional argument: 'criteria'

In [129]:
df1

0                         []
1                         []
2                         []
3      [TX, ADESÃO, -, 1ª/1]
4                         []
               ...          
214                       []
215                       []
216                       []
217                       []
218                       []
Name: 0, Length: 219, dtype: object